loading the essential libraries. pandas and numpy handle data processing, plotly handles visualizations, and LightFM is the core recommendation engine. ast parses the stringified JSON fields in datasets.

In [ ]:
!pip install gradio plotly lightfm

  Using cached lightfm-1.17.tar.gz (316 kB)
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.8 MB/s eta 0:00:00
  Created wheel for lightfm: filename=lightfm-1.17-cp311-cp311-linux_x86_64.whl size=831162 sha256=750051808c618014ac72d09333efd217fb2f66a34138ced70415798e31a8814c
  Stored in directory: /root/.cache/pip/wheels/b9/0d/8a/0729d2e6e3ca2a898ba55201f905da7db3f838a33df5b3fcdd
Successfully built lightfm
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninst

In [ ]:
# ============================
# 📚 Import Libraries
# ============================
import pandas as pd
import numpy as np
import ast
import os
import json
import plotly.express as px
from sklearn.preprocessing import MultiLabelBinarizer
from scipy.sparse import csr_matrix
from lightfm import LightFM
import plotly.io as pio

# Ensure plotly works in Colab
pio.renderers.default = 'colab'


In [ ]:
# ============================
# 📥 Load and Preprocess Data
# ============================
print("📥 Loading Data...")

movies = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Movie Recommender System/tmdb_5000_movies.csv')
credits = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Movie Recommender System/tmdb_5000_credits.csv')

movies = movies.merge(credits, left_on='id', right_on='movie_id').drop('movie_id', axis=1)
movies['title'] = movies['title_x'].fillna(movies['title_y'])
movies = movies.drop(columns=['title_x', 'title_y'])

def parse_genres(genre_str):
    try:
        genres = [g['name'].strip().lower() for g in ast.literal_eval(genre_str)]
        return genres if genres else ['unknown']
    except (ValueError, SyntaxError):
        return ['unknown']

movies['genres'] = movies['genres'].apply(parse_genres)
movies = movies[movies['genres'].apply(lambda g: 'unknown' not in g)]
movies['title'] = movies['title'].str.strip()
movies = movies.drop_duplicates(subset=['id'])

print("✅ Data Loaded & Preprocessed")

# ============================
# 🔗 Genre Features Matrix
# ============================
mlb = MultiLabelBinarizer()
genre_features = pd.DataFrame(mlb.fit_transform(movies['genres']), columns=mlb.classes_, index=movies['id'])

print("✅ Genre Features Matrix Created")

# ============================
# 👤 Simulated User Interactions
# ============================
np.random.seed(42)
user_movie_interactions = []
num_users = 100

for user_id in range(num_users):
    watched_movies = np.random.choice(movies['id'].values, size=np.random.randint(5, 20), replace=False)
    for movie_id in watched_movies:
        user_movie_interactions.append((user_id, movie_id, 1))

interactions_df = pd.DataFrame(user_movie_interactions, columns=['user_id', 'item_id', 'rating'])
movie_id_to_idx = {movie_id: idx for idx, movie_id in enumerate(movies['id'])}
movies['matrix_index'] = movies['id'].map(movie_id_to_idx)
interactions_df['item_id'] = interactions_df['item_id'].map(movie_id_to_idx)

print("✅ Simulated User Interactions Created")

# ============================
# 📊 Interaction & Feature Matrices
# ============================
interaction_matrix = csr_matrix(
    (interactions_df['rating'], (interactions_df['user_id'], interactions_df['item_id'])),
    shape=(num_users, len(movies))
)
item_features = csr_matrix(genre_features.loc[movies['id']].values)

print("✅ Matrices Ready")

# ============================
# 🧠 Train LightFM Model
# ============================
print("🧠 Training LightFM Model...")
model = LightFM(loss='warp')
model.fit(interaction_matrix, item_features=item_features, epochs=30, num_threads=4)
print("✅ Model Trained")

# ============================
# 💾 Persistent Session Tracking
# ============================
session_file = "user_session_memory.json"

def load_session():
    if os.path.exists(session_file):
        with open(session_file, 'r') as f:
            return json.load(f)
    return {}

def save_session(session_data):
    with open(session_file, 'w') as f:
        json.dump(session_data, f)

session_memory = load_session()

# ============================
# ⭐ Trending Movies
# ============================
def recommend_trending_movies_by_genre(genre=None, top_n=10):
    if genre:
        mask = movies['genres'].apply(lambda g: genre.lower() in g)
        genre_movies = movies[mask]
    else:
        genre_movies = movies

    genre_movies['popularity'] = genre_movies['matrix_index'].map(interactions_df['item_id'].value_counts()).fillna(0)
    trending = genre_movies.sort_values('popularity', ascending=False).head(top_n)

    print(f"\n🔥 Trending Movies {'in ' + genre.capitalize() if genre else 'Overall'}:")

    display_df = trending[['title', 'genres', 'popularity']].copy()
    display_df['genres'] = display_df['genres'].apply(lambda g: ', '.join(g))

    print(display_df.to_string(index=False))

# ============================
# 📊 Visualization
# ============================
def visualize_recommendations(user_id, recommended_movies, scores):
    fig = px.bar(
        x=scores,
        y=recommended_movies['title'],
        orientation='h',
        color=recommended_movies['genres'].apply(lambda g: ', '.join(g)),
        title=f"Top {len(recommended_movies)} Recommendations for User {user_id}",
    )
    fig.show()

# ============================
# 🎯 Recommend Movies for User
# ============================
def recommend_for_user(user_id, genre_filter=None, num_recommendations=10):
    if genre_filter is None and str(user_id) in session_memory:
        print(f"\n✅ Using Cached Recommendations for User {user_id}:")
        for idx, title in enumerate(session_memory[str(user_id)], start=1):
            print(f"{idx}. {title}")
        return

    scores = model.predict(user_id, np.arange(len(movies)), item_features=item_features)

    if genre_filter:
        mask = movies['genres'].apply(lambda g: genre_filter.lower() in g)
        scores = np.where(mask, scores, -np.inf)

    recommended_indices = np.argsort(-scores)[:num_recommendations]
    recommended_movies = movies.iloc[recommended_indices]

    session_memory[str(user_id)] = recommended_movies['title'].tolist()
    save_session(session_memory)

    print(f"\n🎯 Recommended Movies for User {user_id}:")
    display_df = recommended_movies[['title', 'genres']].copy()
    display_df['genres'] = display_df['genres'].apply(lambda g: ', '.join(g))
    print(display_df.to_string(index=False))

    visualize_recommendations(user_id, recommended_movies, scores[recommended_indices])

# ============================
# 🎬 Main Interactive Loop
# ============================
print("\n🚀 Movie Recommender Ready!")
print(f"Available User IDs: [0 - {num_users-1}]")

# Ask for trending genre
genre = input("\nEnter a genre for trending movies (or press Enter to show all): ").strip()
recommend_trending_movies_by_genre(genre if genre else None)

# Ask for user ID
user_input = input("\nEnter User ID for personalized recommendations (0-99) or press Enter to skip: ").strip()

if user_input:
    user_id = int(user_input)

    # Ask for genre filter
    genre_filter = input("Filter recommendations by genre (or press Enter for all genres): ").strip()
    recommend_for_user(user_id, genre_filter if genre_filter else None)
else:
    print("✅ Skipped Personalized Recommendations")

print("\n✅ Session Complete!")


📥 Loading Data...


<ipython-input-4-44acf051e83f>:39: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



✅ Data Loaded & Preprocessed
✅ Genre Features Matrix Created
✅ Simulated User Interactions Created
✅ Matrices Ready
🧠 Training LightFM Model...
✅ Model Trained

🚀 Movie Recommender Ready!
Available User IDs: [0 - 99]

Enter a genre for trending movies (or press Enter to show all): comey

🔥 Trending Movies in Comey:
Empty DataFrame
Columns: [title, genres, popularity]
Index: []

Enter User ID for personalized recommendations (0-99) or press Enter to skip: 1
Filter recommendations by genre (or press Enter for all genres): action

🎯 Recommended Movies for User 1:
                           title                                                                 genres
       Jimmy Neutron: Boy Genius action, adventure, animation, comedy, family, fantasy, science fiction
Teenage Mutant Ninja Turtles III            action, adventure, comedy, family, fantasy, science fiction
                    Thunderbirds            action, adventure, comedy, family, fantasy, science fiction
                 


✅ Session Complete!
